Módulo de análisis sintáctico que hace parte de SEÑAL: Programa para el análisis lingüístico y evaluación de composiciones de español.

© Restrepo-Ramos
**2022**

# Módulo de complejidad sintáctica

Este módulo permite medir la complejidad sintáctica de tus ensayos. Primero, el programa divide en unidades sintácticas tus oraciones utilizando un modelo pre-entrenado de español (más info [aquí](https://universaldependencies.org/)). El programa utiliza este modelo como muestra para analizar tus composiciones. Por último, el programa te presenta varias medidas de complejidad sintáctica de tus escritos. 

### Requisitos
No hay requisitos técnicos. Sólo presta atención a la presentación de tu instructor.

### Materiales
1. Módelo pre-entrenado de español ***es.udpipe***
2. Tu ***muestra.txt***

### Instrucciones
1. Ejecuta todas las líneas de código leyendo la descripción de cada funcionalidad del programa
3. interpreta los resultados de acuerdo a las distintas medidas de complejidad sintáctica

**Paso 1**. Instalamos UDPipe

UDPipe es el módulo que nos va a permitir dividir cada oración de tus composiciones en unidades sintácticas (i.e., *parsing*). El programa nos va a arrojar un archivo en formato ***.conllu*** 

In [ ]:
!pip install ufal.udpipe

     |████████████████████████████████| 304 kB 5.2 MB/s 
  Created wheel for ufal.udpipe: filename=ufal.udpipe-1.2.0.3-cp37-cp37m-linux_x86_64.whl size=5626658 sha256=8d077e2c359f22ef9235fcac7bb3297652c926de8fdbe75ab6f41e57c9c94f2d
  Stored in directory: /root/.cache/pip/wheels/b8/b5/8e/3da091629a21ce2d10bf90759d0cb034ba10a5cf7a01e83d64
Successfully built ufal.udpipe


**Paso 2**

Ejecutamos las siguiente línea de código. Aunque parece complicado, está parte del programa crea *definiciones* para que cuando subas tu composición, el programa pueda realizar la división sintáctica.

In [ ]:
import sys
import ufal.udpipe

class Model:
    def __init__(self, path):
        """Load given model."""
        self.model = ufal.udpipe.Model.load(path)
        if not self.model:
            raise Exception("Cannot load UDPipe model from file '%s'" % path)

    def tokenize(self, text):
        """Tokenize the text and return list of ufal.udpipe.Sentence-s."""
        tokenizer = self.model.newTokenizer(self.model.DEFAULT)
        if not tokenizer:
            raise Exception("The model does not have a tokenizer")
        return self._read(text, tokenizer)

    def read(self, text, in_format):
        """Load text in the given format (conllu|horizontal|vertical) and return list of ufal.udpipe.Sentence-s."""
        input_format = ufal.udpipe.InputFormat.newInputFormat(in_format)
        if not input_format:
            raise Exception("Cannot create input format '%s'" % in_format)
        return self._read(text, input_format)

    def _read(self, text, input_format):
        input_format.setText(text)
        error = ufal.udpipe.ProcessingError()
        sentences = []

        sentence = ufal.udpipe.Sentence()
        while input_format.nextSentence(sentence, error):
            sentences.append(sentence)
            sentence = ufal.udpipe.Sentence()
        if error.occurred():
            raise Exception(error.message)

        return sentences

    def tag(self, sentence):
        """Tag the given ufal.udpipe.Sentence (inplace)."""
        self.model.tag(sentence, self.model.DEFAULT)

    def parse(self, sentence):
        """Parse the given ufal.udpipe.Sentence (inplace)."""
        self.model.parse(sentence, self.model.DEFAULT)

    def write(self, sentences, out_format):
        """Write given ufal.udpipe.Sentence-s in the required format (conllu|horizontal|vertical)."""

        output_format = ufal.udpipe.OutputFormat.newOutputFormat(out_format)
        output = ''
        for sentence in sentences:
            output += output_format.writeSentence(sentence)
        output += output_format.finishDocument()

        return output


**Paso 3**

Subimos el módelo preentrenado de español



In [ ]:
model = Model('es.udpipe')

**Paso 4**

Subimos nuestro ensayo a la memoria. Si tienes problemas con MAC cambia el encoding a: ISO-8859-1

In [ ]:
ifile = open("muestra.txt", mode='r', encoding='utf-8')
texto = ifile.read()
ifile.close()

**Paso 5**

Utilizamos el modelo preentrenado y realizamos la división sintáctica de nuestras composiciones (*Parsing*).

In [ ]:
sentences = model.tokenize(texto)
for s in sentences:
    model.tag(s)
    model.parse(s)
conllu = model.write(sentences, "conllu")

Así luce una oración dividida sintácticamente por el programa:

sent_id = 4
text = Por último, el campus no promueve la salud y el bienestar tanto como debería.

1.	Por	por	ADP	_	_	2	case	_	_
2.	último	último	NOUN	_	Gender=Masc|Number=Sing	7	obl	_	SpaceAfter=No
3.	,	,	PUNCT	_	_	2	punct	_	_
4.	el	el	DET	_	Definite=Def|Gender=Masc|Number=Sing|PronType=Art	5	det	_	_
5.	campus	campus	NOUN	_	Gender=Masc	7	nsubj	_	_
6.	no	no	ADV	_	Polarity=Neg	7	advmod	_	_
7.	promueve	promover	VERB	_	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	0	root	_	_
8.	la	el	DET	_	Definite=Def|Gender=Fem|Number=Sing|PronType=Art	9	det	_	_
9.	salud	salud	NOUN	_	Gender=Fem|Number=Sing	7	obj	_	_
10.	y	y	CCONJ	_	_	12	cc	_	_
11.	el	el	DET	_	Definite=Def|Gender=Masc|Number=Sing|PronType=Art	12	det	_	_
12.	bienestar	bienestar	VERB	_	Gender=Masc|Number=Sing|VerbForm=Fin	7	conj	_	_
13.	tanto	tanto	PRON	_	NumType=Card|PronType=Dem	12	obl	_	_
14.	como	como	ADP	_	_	15	case	_	_
15.	debería	debería	NOUN	_	Gender=Fem|Number=Sing	12	obl	_	SpaceAfter=No
16.	.	.	PUNCT	_	_	7	punct	_	_



**Paso 6**

Ahora vamos a contar la frecuencia de los siguientes indicadores sintacticos por oración:

1. promedio de verbos por oración
2. promedio de palabras por oración
3. promedio de verbos auxiliares por oración
4. promedio de cláusulas relativas por oración
5. promedio de cláusulas adverbiales por oración
6. promedio general de cláusulas
7. promedio de t-units
8. promedio de conjunciones coordinativas

Explicaré todo esto en clase.



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# for each of the sentences in the file
v = []
w = []
aux = []
rel = []
adv = []
clauses = []
t = []
conj = []
numsent = 0

for sent in conllu.split('\n\n'): # aquí subimos el archivo .conllu que creamos
    sent_id = '' # the sentence id
    ntokens = 0 # number of words/tokens
    nverbs = 0
    naux = 0
    npunct = 0
    nrelclaus = 0
    finverb = 0
    nconj= 0
    tunits= 0
    advclaus = 0
    comp = 0
    numsent += 1
    if sent.strip() == '': # if there is no data in the sentence then skip it
        continue
    #for each of the lines in the sentence
    for line in sent.split('\n'):
        # if the line contains the string 'sent_id', then sent the sent id to be the part after the '='
        if line.count('sent_id') > 0:
            sent_id = line.split('=')[1].strip()
            # if the line doesn't start with a # then increment the number of words
        if line[0] !='#':
            ntokens += 1
        #row = line.split('\t')
        #if not row[1].outnum():
         #   npunct+=1
        if line.count('\tPUNCT\t') > 0:
            npunct += 1
        if line.count('\tVERB\t') > 0:
            nverbs +=1
        if line.count('\tAUX\t') > 0:
            naux +=1
        if line.count('\tacl:relcl\t') > 0:
            nrelclaus +=1
        if line.count('\tadvcl\t') > 0:
            advclaus += 1
        if line.count('\tccomp\t') > 0:
            comp += 1
        if line.count('\tVerbForm=Fin\t') > 0:
            finverb += 1
        if line.count('\tCCONJ\t') > 0:
            nconj += 1
        if line.count('\tconj\t') > 0 or line.count('\tparataxis\t'):
            tunits += 1

    t_units = tunits  + 1
    nwords = ntokens-npunct
    #nclauses = nverbs + naux + nrelclaus - infverb -conj
    nclauses = nrelclaus + advclaus + t_units

    # print out sentence id, number of words, and verbs per clause. This is for creating a table if desiredfor further analysis
    print('%s\t%d\t%d\t%d\t%d\t%d\t%d\t%d\t%d\t%d\t%d' % (sent_id, nwords, ntokens, npunct, nverbs, naux, nrelclaus, advclaus, nclauses, t_units, nconj))

# here we are attaching the numbers in the counters to specific lists.
    v.append(nverbs)
    w.append(nwords)
    aux.append(naux)
    rel.append(nrelclaus)
    adv.append(advclaus)
    clauses.append(nclauses)
    t.append(t_units)
    conj.append(nconj)

print('sent_id','nwords', 'ntokens', 'npunct', 'nverbs', 'naux', 'nrelclaus', 'advclaus','nclaus', 't_units', 'nconj')
# creating floats for the plots
vnum = [float(i) for i in v]
vmean = (sum(vnum)/(numsent-1))
wnum = [float(i) for i in w]
wmean = (sum(wnum)/(numsent-1))
auxnum = [float(i) for i in aux]
auxmean = (sum(auxnum)/(numsent-1))
relnum = [float(i) for i in rel]
relmean = (sum(relnum)/(numsent-1))
advnum = [float(i) for i in adv]
advmean = (sum(advnum)/(numsent-1))
clausenum = [float(i) for i in clauses]
clausemean = (sum(clausenum)/(numsent-1))
tnum = [float(i) for i in t]
tmean = (sum(tnum)/(numsent-1))
conjnum = [float(i) for i in conj]
conjmean = (sum(conjnum)/(numsent-1))


**Paso 7**

Creamos la tabla con todos los resultados

In [ ]:
#creating a nice printing table
average = [vmean, wmean, auxmean, relmean, advmean, clausemean, tmean, conjmean]
names = ["Verbos", "Palabras", "AUX", "Relativas", "Adverbiales", "Cláusulas", "T-Units", "Coordinación"]
dct = {names[i]: average[i] for i in range(len(names))}

print("\nEste es el promedio de frecuencia de los indicadores sintácticos de tu glosario")
print("Medida:\t\tPromedio de frecuencia:")
for k, v in dct.items():
    print(k, '\t', v)

**Paso 8**

Creamos un hermoso gráfico con el conteo y promedio de todos los indicadores sintácticos por oración.

In [ ]:
data = [vmean, auxmean, relmean, advmean, clausemean, tmean, conjmean, wmean]
fig = plt.figure(figsize =(13, 7))
ax = fig.add_axes([0,0,1,1]) 
indicadores = ['Verbos', 'VerbosAux', 'ClausuRel', 'ClausuAdv', 'Cláusulas', 'T-Units', 'Coordinación', 'Palabras']
ax.barh(indicadores, data)
plt.title('Promedio de indicadores sintácticos')
for index, value in enumerate(data):
    plt.text(value, index,
             str(value))
plt.show(fig)

**Ahora tú**

Completa los espacios necesarios en las siguientes líneas de código para completar tu hermoso gráfico.

In [ ]:
# seleccionamos los datos del eje 'x' e 'y'
x = ['Verbos', 'VerbosAux', 'ClausuRel', 'ClausuAdv', 'Cláusulas', 'T-Units', 'Coordinación', 'Palabras']
y = [vmean, auxmean, relmean, advmean, clausemean, tmean, conjmean, wmean]

# PARA HACER: ajusta el tamaño del gráfico
fig = plt.figure(figsize =(??, ?)) # COMPLETAR

# PARA HACER: ajusta los colores. Puedes escoger más colores aquí: https://matplotlib.org/stable/gallery/color/named_colors.html
colors = ['gold', 'red', 'blue','slategray','hotpink', 'orchid', 'lime', 'deepskyblue']
plt.bar(x, y, color=colors)

#PARA HACER: Escoge un título para tu gráfico
plt.title('Mi complejidad sintáctica en español') # tu título aquí

# PARA HACER =¿Cuántos decimales quieres en cada barra?
for index, value in enumerate(y):
    plt.text(index, value, 
             str(round(value, 3))) # ajusta el número de decimales

# muestra el gráfic0
plt.show(fig)